# UC7 with Attack Variability

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

This notebook demonstrates the PrimAITE environment with the UC7 network laydown and multiple attack personas. The first threat actor persona is **TAP001** which performs a ransomware attack against the database. The other one is **TAP003** which is able to maliciously add ACL rules that block green pattern of life.

Any users unfamiliar with these red agents should take a look into the [TAP001 notebook](./UC7-TAP001-Kill-Chain-E2E.ipynb) and the [TAP003 notebook](./UC7-TAP003-Kill-Chain-E2E.ipynb) for further details.

The environment switches between these two attacks on a pre-defined schedule which is defined in the `schedule.yaml` file of the scenario folder.

## Setup and Imports

In [1]:
!primaite setup

2025-03-24 09:59:59,028: Performing the PrimAITE first-time setup...
2025-03-24 09:59:59,029: Building the PrimAITE app directories...
2025-03-24 09:59:59,029: Building primaite_config.yaml...
2025-03-24 09:59:59,029: Rebuilding the demo notebooks...
2025-03-24 09:59:59,052: Rebuilding the example notebooks...
2025-03-24 09:59:59,054: PrimAITE setup complete!


In [2]:
from primaite.session.environment import PrimaiteGymEnv
from primaite import PRIMAITE_PATHS
from prettytable import PrettyTable
from deepdiff.diff import DeepDiff
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.network.hardware.nodes.network.router import Router

scenario_path = PRIMAITE_PATHS.user_config_path / "example_config/uc7_multiple_attack_variants"

In [3]:
env = PrimaiteGymEnv(env_config=scenario_path)

2025-03-24 10:00:04,155: PrimaiteGymEnv RNG seed = None


## Schedule

Let's print the schedule so that we can see which attack we can expect on each episode.

On episodes 0-4, the TAP001 agent will be used, and on episodes 5-9, the TAP003 agent will be used. Then, the environment will alternate between the two. Furthermore, the TAP001 agent will alternate between starting at `ST_PROJ-A-PRV-PC-1`, `ST_PROJ-B-PRV-PC-2`, `ST_PROJ-C-PRV-PC-3`.

In [4]:
with open(scenario_path / "schedule.yaml",'r') as f:
    print(f.read())

base_scenario: uc7_config_no_red.yaml
schedule:
  0:
    - TAP001_PC1.yaml
  1:
    - TAP001_PC2.yaml
  2:
    - TAP001_PC3.yaml
  3:
    - TAP001_PC1.yaml
  4:
    - TAP001_PC2.yaml
  5:
    - TAP003.yaml
  6:
    - TAP003.yaml
  7:
    - TAP003.yaml
  8:
    - TAP003.yaml
  9:
    - TAP003.yaml
  10:
    - TAP001_PC1.yaml
  11:
    - TAP003.yaml
  12:
    - TAP001_PC1.yaml
  13:
    - TAP003.yaml
  14:
    - TAP001_PC2.yaml
  15:
    - TAP003.yaml
  16:
    - TAP001_PC3.yaml
  17:
    - TAP003.yaml
  18:
    - TAP001_PC1.yaml
  19:
    - TAP003.yaml



## TAP001 attack

Let's first demonstrate the TAP001 attack. We will let the environment run for 30 steps and print out the red agent's actions.


In [5]:
#utils
def run_green_and_red_pol(num_steps):
    for i in range(num_steps): # perform steps
        env.step(0)

def print_agent_actions_except_do_nothing(agent_name):
    """Get the agent's action history, filter out `do-nothing` actions, print relevant data in a table."""
    table = PrettyTable()
    table.field_names = ["Step", "Action", "Node", "Application", "Target IP", "Response"]
    print(f"Episode: {env.episode_counter}, Actions for '{agent_name}':")
    for item in env.game.agents[agent_name].history:
        if item.action == "do-nothing":
            continue

        node, application, target_ip = "N/A", "N/A", "N/A",

        if item.action.startswith("node-nmap"):
            node = item.parameters['source_node']
            application = "nmap"
            target_ip = str(item.parameters['target_ip_address'])
            target_ip = (target_ip[:25]+'...') if len(target_ip)>25 else target_ip # truncate long string

        elif item.action == "router-acl-add-rule":
            node = item.parameters.get("router_name")
        elif item.action == "node-send-remote-command":
            node = item.parameters.get("node_name")
            target_ip = item.parameters.get("remote_ip")
            application = item.parameters.get("command")
        elif item.action == "node-session-remote-login":
            node = item.parameters.get("node_name")
            target_ip = item.parameters.get("remote_ip")
            application = "user-manager"
        elif item.action.startswith("c2-server"):
            application = "c2-server"
            node = item.parameters.get('node_name')
        elif item.action == "configure-c2-beacon":
            application = "c2-beacon"
            node = item.parameters.get('node_name')

        else:
            if (node_id := item.parameters.get('node_id')) is not None:
                node = env.game.agents[agent_name].action_manager.node_names[node_id]
            if (application_id := item.parameters.get('application_id')) is not None:
                application = env.game.agents[agent_name].action_manager.application_names[node_id][application_id]
            if (application_name := item.parameters.get('application_name')) is not None:
                application = application_name

        table.add_row([item.timestep, item.action, node, application, target_ip, item.response.status])

    print(table)
    print("(Any do-nothing actions are omitted)")

def finish_episode_and_print_reward():
    while env.game.step_counter < 128:
        env.step(0)
    print(f"Total reward this episode: {env.agent.reward_function.total_reward:2f}")



In [6]:
run_green_and_red_pol(110)
print_agent_actions_except_do_nothing("attacker")

Episode: 0, Actions for 'attacker':
+------+--------------------------------+--------------------+-------------------+------------------+----------+
| Step |             Action             |        Node        |    Application    |    Target IP     | Response |
+------+--------------------------------+--------------------+-------------------+------------------+----------+
|  6   |       node-folder-create       |        N/A         |        N/A        |       N/A        | success  |
|  11  |        node-file-create        |        N/A         |        N/A        |       N/A        | success  |
|  16  |        node-file-access        |        N/A         |        N/A        |       N/A        | success  |
|  21  |    node-application-install    |        N/A         | ransomware-script |       N/A        | success  |
|  26  |      node-nmap-ping-scan       | ST_PROJ-C-PRV-PC-1 |        nmap       | 192.168.230.0/29 | success  |
|  31  |   node-network-service-recon   |        N/A        

In [7]:
st_data_prv_srv_db: Server = env.game.simulation.network.get_node_by_hostname("ST_DATA-PRV-SRV-DB")
st_data_prv_srv_db.file_system.show()

+----------------------------------------------------------------------+
|                    ST_DATA-PRV-SRV-DB File System                    |
+----------+---------+---------------+-----------------------+---------+
| Folder   | Size    | Health status | Visible health status | Deleted |
+----------+---------+---------------+-----------------------+---------+
| database | 4.77 MB | CORRUPT       | NONE                  | False   |
| root     | 0 B     | GOOD          | NONE                  | False   |
+----------+---------+---------------+-----------------------+---------+


In [8]:
finish_episode_and_print_reward()

Total reward this episode: 140.284375


## TAP001 Prevention

The blue agent should be able to prevent the ransomware attack by blocking the red agent's access to the database. Let's run the environment until the observation space shows symptoms of the attack starting.

Because we are in episode index 1, the red agent will use `ST_PROJ-A-PRV-PC-1` to start the attack. On step 25, the red agent installs `ransomware-script`.

In [9]:
env.reset()
obs, reward, term, trunc, info = env.step(0)
for i in range(25): # we know that the ransomware install happens at step 25
    old = obs
    obs, reward, term, trunc, info = env.step(0)
    new = obs

diff = DeepDiff(old,new)
print(f"Step {env.game.step_counter}") # it's step 26 now because the step counter is incremented after the step
for d,v in diff.get('values_changed', {}).items():
    print(f"{d}: {v['old_value']} -> {v['new_value']}")


2025-03-24 10:00:16,490: Resetting environment, episode 0, avg. reward: 140.28437499999995


2025-03-24 10:00:16,493: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_0.json


Step 26
root['NODES']['HOST0']['NICS'][1]['TRAFFIC']['tcp'][80]['inbound']: 0 -> 1
root['NODES']['HOST0']['NICS'][1]['TRAFFIC']['tcp'][80]['outbound']: 0 -> 1
root['NODES']['HOST2']['NICS'][1]['TRAFFIC']['tcp'][80]['inbound']: 1 -> 0
root['NODES']['HOST2']['NICS'][1]['TRAFFIC']['tcp'][80]['outbound']: 1 -> 0
root['LINKS'][1]['PROTOCOLS']['ALL']: 0 -> 1
root['LINKS'][5]['PROTOCOLS']['ALL']: 0 -> 1
root['LINKS'][20]['PROTOCOLS']['ALL']: 1 -> 0
root['LINKS'][26]['PROTOCOLS']['ALL']: 1 -> 0


We can see that on HOST0, application index 1 has gone from `operating_status` 0 to 3, meaning there wasn't an application before, but now there is an application in the `INSTALLING` state. The blue agent should be able to detect this and block the red agent's access to the database. Action 43 will block `ST_PROJ-A-PRV-PC-1` from sending POSTGRES traffic to the DB server.

If this were a different episode, it could have been `ST_PROJ-B-PRV-PC-2` or `ST_PROJ-C-PRV-PC-3` that are affected, and a different defensive action would be required.


```yaml

# ST_INTRA-PRV-RT-CR | router-acl-add-rule | P2: ST_PROJ-A-PRV-PC-1 !==> ST_DATA-PRV-SRV-DB (TCP:POSTGRES_SERVER)
43:
    action: router-acl-add-rule
    options:
    target_router: ST_INTRA-PRV-RT-CR
    position: 1
    permission: DENY
    src_ip: 192.168.230.2 # (ST_PROJ-A-PRV-PC-1)
    src_wildcard: 0.0.255.255
    src_port: POSTGRES_SERVER
    dst_ip: 192.168.220.3 # (ST_DATA-PRV-SRV-DB)
    dst_wildcard: 0.0.255.255
    dst_port: POSTGRES_SERVER
    protocol_name: TCP
```

In [10]:
env.step(43);

```yaml

# ST_INTRA-PRV-RT-CR | router-acl-add-rule | P3: ST_PROJ-B-PRV-PC-2 !==> ST_DATA-PRV-SRV-DB (TCP:POSTGRES_SERVER)
45:
  action: router-acl-add-rule
  options:
    target_router: ST_INTRA-PRV-RT-CR
    position: 2
    permission: DENY
    src_ip: 192.168.240.3 # (ST_PROJ-B-PRV-PC-2)
    src_wildcard: 0.0.255.255
    src_port: POSTGRES_SERVER
    dst_ip: 192.168.220.3 # (ST_DATA-PRV-SRV-DB)
    dst_wildcard: 0.0.255.255
    dst_port: POSTGRES_SERVER
    protocol_name: TCP

```

In [11]:
env.step(45);

```yaml

# ST_INTRA-PRV-RT-CR | router-acl-add-rule | P4: ST_PROJ-C-PRV-PC-3 !==> ST_DATA-PRV-SRV-DB (TCP:POSTGRES_SERVER)
47:
  action: router-acl-add-rule
  options:
    target_router: ST_INTRA-PRV-RT-CR
    position: 3
    permission: DENY
    src_ip: 192.168.250.4 # (ST_PROJ-C-PRV-PC-3)
    src_wildcard: 0.0.255.255
    src_port: POSTGRES_SERVER
    dst_ip: 192.168.220.3 # (ST_DATA-PRV-SRV-DB)
    dst_wildcard: 0.0.255.255
    dst_port: POSTGRES_SERVER
    protocol_name: TCP
          
```

In [12]:
env.step(47);

In [13]:
st_intra_prv_rt_cr: Router = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR")
st_intra_prv_rt_cr.acl.show()

+-------------------------------------------------------------------------------------------------------------------------+
|                                          ST_INTRA-PRV-RT-CR Access Control List                                         |
+-------+--------+----------+---------------+--------------+----------+---------------+--------------+----------+---------+
| Index | Action | Protocol | Src IP        | Src Wildcard | Src Port | Dst IP        | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+---------------+--------------+----------+---------------+--------------+----------+---------+
| 1     | DENY   | tcp      | 192.168.230.2 | 0.0.255.255  | 5432     | 192.168.220.3 | 0.0.255.255  | 5432     | 5       |
| 2     | DENY   | tcp      | 192.168.240.3 | 0.0.255.255  | 5432     | 192.168.220.3 | 0.0.255.255  | 5432     | 0       |
| 3     | DENY   | tcp      | 192.168.250.4 | 0.0.255.255  | 5432     | 192.168.220.3 | 0.0.255.255  | 5432     | 0       |
| 5     

In [14]:
finish_episode_and_print_reward()

Total reward this episode: 130.568750


In [15]:
st_intra_prv_rt_cr.acl.show()

+-------------------------------------------------------------------------------------------------------------------------+
|                                          ST_INTRA-PRV-RT-CR Access Control List                                         |
+-------+--------+----------+---------------+--------------+----------+---------------+--------------+----------+---------+
| Index | Action | Protocol | Src IP        | Src Wildcard | Src Port | Dst IP        | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+---------------+--------------+----------+---------------+--------------+----------+---------+
| 1     | DENY   | tcp      | 192.168.230.2 | 0.0.255.255  | 5432     | 192.168.220.3 | 0.0.255.255  | 5432     | 336     |
| 2     | DENY   | tcp      | 192.168.240.3 | 0.0.255.255  | 5432     | 192.168.220.3 | 0.0.255.255  | 5432     | 0       |
| 3     | DENY   | tcp      | 192.168.250.4 | 0.0.255.255  | 5432     | 192.168.220.3 | 0.0.255.255  | 5432     | 0       |
| 5     

Now TAP001 is unable to locate the database!

In [16]:
print_agent_actions_except_do_nothing("attacker")

Episode: 1, Actions for 'attacker':
+------+----------------------------+--------------------+-------------------+------------------+----------+
| Step |           Action           |        Node        |    Application    |    Target IP     | Response |
+------+----------------------------+--------------------+-------------------+------------------+----------+
|  6   |     node-folder-create     |        N/A         |        N/A        |       N/A        | success  |
|  11  |      node-file-create      |        N/A         |        N/A        |       N/A        | success  |
|  16  |      node-file-access      |        N/A         |        N/A        |       N/A        | success  |
|  21  |  node-application-install  |        N/A         | ransomware-script |       N/A        | success  |
|  26  |    node-nmap-ping-scan     | ST_PROJ-B-PRV-PC-2 |        nmap       | 192.168.240.0/29 | success  |
|  31  | node-network-service-recon |        N/A         |        N/A        |       N/A    

## TAP003 attack

Let's skip until episode 5 and demonstrate the TAP003 attack. We will let the environment run and print out the red agent's actions.

By default, TAP003 will add the following rules:

|Target Router         | Impact |
|----------------------|--------|
|`ST_INTRA-PRV-RT-DR-1`| Blocks all `POSTGRES_SERVER` that arrives at the `ST_INTRA-PRV-RT-DR-1` router. This rule will prevent all ST_PROJ_* hosts from accessing the database (`ST_DATA-PRV-SRV-DB`).|
|`ST_INTRA-PRV-RT-CR`| Blocks all `HTTP` traffic that arrives at the`ST_INTRA-PRV-RT-CR` router. This rule will prevent all SOME_TECH hosts from accessing the webserver (`ST_DMZ-PUB-SRV-WEB`)|
|`REM-PUB-RT-DR`| Blocks all `DNS` traffic that arrives at the `REM-PUB-RT-DR` router. This rule prevents any remote site works from accessing the DNS Server (`ISP-PUB-SRV-DNS`).|

In [17]:
while env.episode_counter < 5:
    env.reset()

2025-03-24 10:00:37,595: Resetting environment, episode 1, avg. reward: 130.56874999999968


2025-03-24 10:00:37,598: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_1.json


2025-03-24 10:00:38,338: Resetting environment, episode 2, avg. reward: 0.0


2025-03-24 10:00:38,339: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_2.json


2025-03-24 10:00:39,057: Resetting environment, episode 3, avg. reward: 0.0


2025-03-24 10:00:39,058: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_3.json


2025-03-24 10:00:39,510: Resetting environment, episode 4, avg. reward: 0.0


2025-03-24 10:00:39,511: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_4.json


In [18]:
run_green_and_red_pol(128)
print_agent_actions_except_do_nothing("attacker")
obs, reward, term, trunc, info = env.step(0); # one more step so we can capture the value of `obs`

Episode: 5, Actions for 'attacker':
+------+---------------------------+--------------------+-----------------------------------------------------------------------------------------------+---------------+----------+
| Step |           Action          |        Node        |                                          Application                                          |   Target IP   | Response |
+------+---------------------------+--------------------+-----------------------------------------------------------------------------------------------+---------------+----------+
|  13  | node-session-remote-login | ST_PROJ-A-PRV-PC-1 |                                          user-manager                                         | 192.168.230.1 | success  |
|  16  |  node-send-remote-command | ST_PROJ-A-PRV-PC-1 |          ['service', 'user-manager', 'change_password', 'admin', 'admin', 'red_pass']         | 192.168.230.1 | success  |
|  19  | node-session-remote-login | ST_PROJ-A-PRV-PC-1 |  

The agent selected to add ACL rules that will prevent green pattern of life by blocking a variety of different traffic. This has a negative impact on reward. Let's view the ACL list on the affected router.

In [19]:
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1").acl.show()

+-----------------------------------------------------------------------------------------------------------+
|                                  ST_INTRA-PRV-RT-DR-1 Access Control List                                 |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 1     | DENY   | tcp      | ANY    | 0.0.255.255  | 5432     | ANY    | 0.0.255.255  | 5432     | 218     |
| 5     | PERMIT | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 639     |
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | 

In [20]:
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR").acl.show()

+-----------------------------------------------------------------------------------------------------------+
|                                   ST_INTRA-PRV-RT-CR Access Control List                                  |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 1     | DENY   | tcp      | ANY    | 0.0.255.255  | 80       | ANY    | 0.0.255.255  | 80       | 557     |
| 5     | PERMIT | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 933     |
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | 

In [21]:
env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR").acl.show()

+-----------------------------------------------------------------------------------------------------------+
|                                     REM-PUB-RT-DR Access Control List                                     |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 1     | DENY   | tcp      | ANY    | 0.0.255.255  | 53       | ANY    | 0.0.255.255  | 53       | 0       |
| 5     | PERMIT | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 235     |
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | 

We can see that at indices 1-5, there are ACL rules that block all traffic. The blue agent can see this rule in the `ROUTERS` part of the observation space.


In [22]:
obs['NODES']['ROUTER0']['ACL'][1]

{'position': 1,
 'permission': 2,
 'source_ip_id': 1,
 'source_wildcard_id': 4,
 'source_port_id': 4,
 'dest_ip_id': 1,
 'dest_wildcard_id': 4,
 'dest_port_id': 4,
 'protocol_id': 3}

In [23]:
obs['NODES']['ROUTER1']['ACL'][1]

{'position': 1,
 'permission': 2,
 'source_ip_id': 1,
 'source_wildcard_id': 4,
 'source_port_id': 6,
 'dest_ip_id': 1,
 'dest_wildcard_id': 4,
 'dest_port_id': 6,
 'protocol_id': 3}

In [24]:
obs['NODES']['ROUTER2']['ACL'][1]

{'position': 1,
 'permission': 2,
 'source_ip_id': 1,
 'source_wildcard_id': 4,
 'source_port_id': 3,
 'dest_ip_id': 1,
 'dest_wildcard_id': 4,
 'dest_port_id': 3,
 'protocol_id': 3}

## Preventing TAP003 attack

TAP003 relies on connecting to the routers via SSH, and sending `add_rule` terminal commands. The blue agent can prevent this by pre-emptively changing the admin password on the affected routers or by blocking SSH traffic between the red agent's starting node and the target routers.

By printing the reward of each individual agent, we will see what green agents are affected the most. Of course, these green rewards count towards the blue reward so ultimately the blue agent should learn to remove the ACL rule.

In [25]:
env.reset()
finish_episode_and_print_reward()
for ag in env.game.agents.values():
    print(ag.config.ref, ag.reward_function.total_reward)

2025-03-24 10:00:46,530: Resetting environment, episode 5, avg. reward: 117.67656249999963


2025-03-24 10:00:46,532: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_5.json


Total reward this episode: 116.559375
HOME_WORKER-1-DB 61.5
HOME_WORKER-1-WEB 25.59999999999994
HOME_WORKER-2-DB 60.5
HOME_WORKER-2-WEB 25.39999999999994
REMOTE_WORKER-1-DB 57.5
REMOTE_WORKER-1-WEB 25.199999999999942
REMOTE_WORKER-2-DB 56.0
REMOTE_WORKER-2-WEB 24.999999999999943
PROJ_A-SENIOR-DEV-DB -29.0
PROJ_A-SENIOR-DEV-WEB -9.6
PROJ_A-JUNIOR-DEV-1-DB -30.0
PROJ_A-JUNIOR-DEV-1-WEB -9.200000000000001
PROJ_A-JUNIOR-DEV-2-DB -29.0
PROJ_A-JUNIOR-DEV-2-WEB -9.4
PROJ_B-SENIOR-DEV-DB -29.0
PROJ_B-SENIOR-DEV-WEB -10.199999999999998
PROJ_B-JUNIOR-DEV-1-DB -28.0
PROJ_B-JUNIOR-DEV-1-WEB -6.200000000000005
PROJ_B-JUNIOR-DEV-2-DB -29.5
PROJ_B-JUNIOR-DEV-2-WEB -10.799999999999995
PROJ_C-SENIOR-DEV-DB -26.0
PROJ_C-SENIOR-DEV-WEB -7.2000000000000055
PROJ_C-JUNIOR-DEV-1-DB -26.5
PROJ_C-JUNIOR-DEV-1-WEB -5.0000000000000036
PROJ_C-JUNIOR-DEV-2-DB -28.0
PROJ_C-JUNIOR-DEV-2-WEB -8.800000000000002
CEO -39.900000000000006
CTO -22.0
CFO -22.5
SENIOR_HR -23.5
JUNIOR_HR-1 -23.0
JUNIOR_HR-2 -23.5
attacker 0.0

The most effective option that the blue agent has against TAP003 is to prevent the red agent from ever adding the ACLs in the first place through blocking the SSH connection.

In [26]:
env.reset()
env.step(51) # SSH Blocking ACL on ST_INRA-PRV-RT-R1

2025-03-24 10:00:53,687: Resetting environment, episode 6, avg. reward: 116.55937499999965


2025-03-24 10:00:53,690: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_6.json


({'NODES': {'HOST0': {'SERVICES': {1: {'operating_status': 2,
      'health_status': 0},
     2: {'operating_status': 0, 'health_status': 0}},
    'APPLICATIONS': {1: {'operating_status': 0,
      'health_status': 0,
      'num_executions': 0},
     2: {'operating_status': 1, 'health_status': 0, 'num_executions': 1}},
    'FOLDERS': {1: {'health_status': 0,
      'FILES': {1: {'health_status': 0, 'num_access': 0}}},
     2: {'health_status': 0,
      'FILES': {1: {'health_status': 0, 'num_access': 0}}}},
    'NICS': {1: {'nic_status': 1,
      'TRAFFIC': {'tcp': {80: {'inbound': 1, 'outbound': 1},
        5432: {'inbound': 1, 'outbound': 1}},
       'icmp': {'inbound': 0, 'outbound': 0}},
      'NMNE': {'inbound': 0, 'outbound': 0}}},
    'num_file_creations': 0,
    'num_file_deletions': 0,
    'users': {'local_login': 0, 'remote_sessions': 0},
    'operating_status': 1},
   'HOST1': {'SERVICES': {1: {'operating_status': 2, 'health_status': 0},
     2: {'operating_status': 0, 'health_

```yaml

# ST_INTRA-PRV-RT-DR-1 | router-acl-add-rule | P1: ST_INTRA-PRV-RT-DR-1 !==> ANY (TCP:SSH)
51:
  action: router-acl-add-rule
  options:
    target_router: ST_INTRA-PRV-RT-DR-1
    position: 1
    permission: DENY
    src_ip: 192.168.230.2 # (ST_PROJ-A-PRV-PC-1)
    src_wildcard: 0.0.255.255
    src_port: SSH
    dst_ip: ALL
    dst_wildcard: 0.0.255.255
    dst_port: SSH
    protocol_name: TCP

```

In [27]:
finish_episode_and_print_reward()

for ag in env.game.agents.values():
    print(ag.config.ref, ag.reward_function.total_reward)

Total reward this episode: 170.906250
HOME_WORKER-1-DB 62.0
HOME_WORKER-1-WEB 24.999999999999943
HOME_WORKER-2-DB 59.5
HOME_WORKER-2-WEB 25.59999999999994
REMOTE_WORKER-1-DB 58.5
REMOTE_WORKER-1-WEB 25.39999999999994
REMOTE_WORKER-2-DB 56.0
REMOTE_WORKER-2-WEB 25.59999999999994
PROJ_A-SENIOR-DEV-DB 64.0
PROJ_A-SENIOR-DEV-WEB 25.59999999999994
PROJ_A-JUNIOR-DEV-1-DB 64.0
PROJ_A-JUNIOR-DEV-1-WEB 24.599999999999945
PROJ_A-JUNIOR-DEV-2-DB 63.0
PROJ_A-JUNIOR-DEV-2-WEB 25.59999999999994
PROJ_B-SENIOR-DEV-DB 63.5
PROJ_B-SENIOR-DEV-WEB 25.59999999999994
PROJ_B-JUNIOR-DEV-1-DB 63.5
PROJ_B-JUNIOR-DEV-1-WEB 25.199999999999942
PROJ_B-JUNIOR-DEV-2-DB 63.5
PROJ_B-JUNIOR-DEV-2-WEB 25.59999999999994
PROJ_C-SENIOR-DEV-DB 63.0
PROJ_C-SENIOR-DEV-WEB 25.59999999999994
PROJ_C-JUNIOR-DEV-1-DB 63.5
PROJ_C-JUNIOR-DEV-1-WEB 24.799999999999944
PROJ_C-JUNIOR-DEV-2-DB 64.0
PROJ_C-JUNIOR-DEV-2-WEB 25.59999999999994
CEO 121.60000000000025
CTO 62.5
CFO 64.0
SENIOR_HR 64.0
JUNIOR_HR-1 64.0
JUNIOR_HR-2 63.5
attacker 0

Additionally, another option the blue agent can take is to change the passwords of the different target routers that TAP003 will attack through the `node-account-change-password` action.

In [28]:
env.reset()

2025-03-24 10:01:02,675: Resetting environment, episode 7, avg. reward: 170.9062499999996


2025-03-24 10:01:02,678: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_7.json


({'NODES': {'HOST0': {'SERVICES': {1: {'operating_status': 2,
      'health_status': 0},
     2: {'operating_status': 0, 'health_status': 0}},
    'APPLICATIONS': {1: {'operating_status': 0,
      'health_status': 0,
      'num_executions': 0},
     2: {'operating_status': 1, 'health_status': 0, 'num_executions': 0}},
    'FOLDERS': {1: {'health_status': 0,
      'FILES': {1: {'health_status': 0, 'num_access': 0}}},
     2: {'health_status': 0,
      'FILES': {1: {'health_status': 0, 'num_access': 0}}}},
    'NICS': {1: {'nic_status': 1,
      'TRAFFIC': {'tcp': {80: {'inbound': 0, 'outbound': 0},
        5432: {'inbound': 0, 'outbound': 0}},
       'icmp': {'inbound': 0, 'outbound': 0}},
      'NMNE': {'inbound': 0, 'outbound': 0}}},
    'num_file_creations': 0,
    'num_file_deletions': 0,
    'users': {'local_login': 0, 'remote_sessions': 0},
    'operating_status': 1},
   'HOST1': {'SERVICES': {1: {'operating_status': 2, 'health_status': 0},
     2: {'operating_status': 0, 'health_

```yaml

# ST_DATA-PRV-SRV-DB | node-account-change-password | Changes the password of a user account
50:
    action: node-account-change-password
    options:
    node_name: ST_DATA-PRV-SRV-DB
    username: admin   # default account
    current_password: admin   # default password
    new_password: thr33_alert_wolv3z # A more 'secure' password
    
```


In [29]:
env.step(50); 

```yaml

# ST_INTRA-PRV-RT-DR-1 | node-account-change-password
52:
    action: node-account-change-password
    options:
    node_name: ST_INTRA-PRV-RT-DR-1
    username: admin
    current_password: admin
    new_password: secure_password

```

In [30]:
env.step(52);

```yaml

# REM-PUB-RT-DR | node-account-change-password
54:
    action: node-account-change-password
    options:
    node_name: REM-PUB-RT-DR
    username: admin
    current_password: admin
    new_password: secure_password

```

In [31]:
env.step(54); 

In [32]:
finish_episode_and_print_reward()

for ag in env.game.agents.values():
    print(ag.config.ref, ag.reward_function.total_reward)

Total reward this episode: 170.606250
HOME_WORKER-1-DB 61.5
HOME_WORKER-1-WEB 24.199999999999946
HOME_WORKER-2-DB 60.5
HOME_WORKER-2-WEB 25.59999999999994
REMOTE_WORKER-1-DB 58.5
REMOTE_WORKER-1-WEB 24.599999999999945
REMOTE_WORKER-2-DB 56.5
REMOTE_WORKER-2-WEB 25.39999999999994
PROJ_A-SENIOR-DEV-DB 63.0
PROJ_A-SENIOR-DEV-WEB 25.39999999999994
PROJ_A-JUNIOR-DEV-1-DB 64.0
PROJ_A-JUNIOR-DEV-1-WEB 24.799999999999944
PROJ_A-JUNIOR-DEV-2-DB 64.0
PROJ_A-JUNIOR-DEV-2-WEB 25.39999999999994
PROJ_B-SENIOR-DEV-DB 63.0
PROJ_B-SENIOR-DEV-WEB 24.999999999999943
PROJ_B-JUNIOR-DEV-1-DB 63.0
PROJ_B-JUNIOR-DEV-1-WEB 25.59999999999994
PROJ_B-JUNIOR-DEV-2-DB 63.0
PROJ_B-JUNIOR-DEV-2-WEB 22.99999999999995
PROJ_C-SENIOR-DEV-DB 63.5
PROJ_C-SENIOR-DEV-WEB 25.59999999999994
PROJ_C-JUNIOR-DEV-1-DB 64.0
PROJ_C-JUNIOR-DEV-1-WEB 25.59999999999994
PROJ_C-JUNIOR-DEV-2-DB 64.0
PROJ_C-JUNIOR-DEV-2-WEB 22.99999999999995
CEO 121.60000000000025
CTO 63.5
CFO 63.0
SENIOR_HR 64.0
JUNIOR_HR-1 64.0
JUNIOR_HR-2 63.5
attacker 0

Lastly, the blue agent can remedy the impacts of TAP003 through removing the malicious ACLs that TAP003 adds.

In [33]:
env.reset()

# Allow TAP003 to add it's malicious rules
for _ in range(45):
    env.step(0)

env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR").acl.show()
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1").acl.show()
env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR").acl.show()

2025-03-24 10:01:11,539: Resetting environment, episode 8, avg. reward: 170.60624999999962


2025-03-24 10:01:11,542: Saving agent action log to /home/runner/primaite/4.0.0/sessions/2025-03-24/09-59-59/agent_actions/episode_8.json


+-----------------------------------------------------------------------------------------------------------+
|                                   ST_INTRA-PRV-RT-CR Access Control List                                  |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 1     | DENY   | tcp      | ANY    | 0.0.255.255  | 80       | ANY    | 0.0.255.255  | 80       | 21      |
| 5     | PERMIT | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 758     |
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | 

```yaml

```

In [34]:
env.step(44);

```yaml

# ST_INTRA-PRV-RT-CR | REMOVE_ACL_ADDRULE | Removes a given ACL at position 1
44:
    action: router-acl-remove-rule
    options:
        target_router: ST_INTRA-PRV-RT-CR
        position: 1

```

In [35]:
env.step(53);

```yaml

# ST_INTRA-PRV-RT-DR-1 | router-acl-remove-rule | Removes the given ACL at position 1
53:
    action: router-acl-remove-rule
    options:
        target_router: ST_INTRA-PRV-RT-DR-1
        position: 1

```

In [36]:
env.step(55);

```yaml

# REM-PUB-RT-DR | router-acl-remove-rule | Removes the given ACL at position 1
55:
    action: router-acl-remove-rule
    options:
        target_router: REM-PUB-RT-DR
        position: 1

```

In [37]:
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR").acl.show()
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1").acl.show()
env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR").acl.show()

+-----------------------------------------------------------------------------------------------------------+
|                                   ST_INTRA-PRV-RT-CR Access Control List                                  |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 5     | PERMIT | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 802     |
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | DENY   | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
+-------+-

In [38]:
finish_episode_and_print_reward()

for ag in env.game.agents.values():
    print(ag.config.ref, ag.reward_function.total_reward)

Total reward this episode: 165.715625
HOME_WORKER-1-DB 62.5
HOME_WORKER-1-WEB 25.39999999999994
HOME_WORKER-2-DB 60.0
HOME_WORKER-2-WEB 25.199999999999942
REMOTE_WORKER-1-DB 57.5
REMOTE_WORKER-1-WEB 24.999999999999943
REMOTE_WORKER-2-DB 56.5
REMOTE_WORKER-2-WEB 24.599999999999945
PROJ_A-SENIOR-DEV-DB 52.5
PROJ_A-SENIOR-DEV-WEB 22.399999999999952
PROJ_A-JUNIOR-DEV-1-DB 51.5
PROJ_A-JUNIOR-DEV-1-WEB 21.399999999999956
PROJ_A-JUNIOR-DEV-2-DB 50.5
PROJ_A-JUNIOR-DEV-2-WEB 24.599999999999945
PROJ_B-SENIOR-DEV-DB 51.0
PROJ_B-SENIOR-DEV-WEB 22.79999999999995
PROJ_B-JUNIOR-DEV-1-DB 47.5
PROJ_B-JUNIOR-DEV-1-WEB 25.39999999999994
PROJ_B-JUNIOR-DEV-2-DB 54.0
PROJ_B-JUNIOR-DEV-2-WEB 20.799999999999958
PROJ_C-SENIOR-DEV-DB 47.5
PROJ_C-SENIOR-DEV-WEB 23.999999999999947
PROJ_C-JUNIOR-DEV-1-DB 52.0
PROJ_C-JUNIOR-DEV-1-WEB 23.39999999999995
PROJ_C-JUNIOR-DEV-2-DB 54.0
PROJ_C-JUNIOR-DEV-2-WEB 21.799999999999955
CEO 112.10000000000022
CTO 58.5
CFO 60.0
SENIOR_HR 59.0
JUNIOR_HR-1 60.0
JUNIOR_HR-2 59.5
attac